O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

## Preparação de Dados

## Treinamento do modelo

## Análise do modelo

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [ ]  O código está livre de erros
- [ ]  As células com o código foram organizadas em ordem de execução
- [ ]  Os dados foram baixados e preparados
- [ ]  Os modelos foram treinados
- [ ]  A análise de velocidade e qualidade dos modelos foi realizada